<a href="https://colab.research.google.com/github/VangatiManikanthReddy/PhishingEmailDetectionUsingLLM/blob/testbranch/DeepseekClassificationFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup and acess

In [ ]:
#  Check Python version
!python --version

Python 3.11.12


In [ ]:
import numpy as np
import pandas as pd
from google.colab import userdata
import os
from google.colab import data_table
from vega_datasets import data


In [ ]:


if not os.getenv("DEEPSEEK_API_KEY"):
    # os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter your deep seek Api")
    os.environ["DEEPSEEK_API_KEY"] = "sk-e3ee19d2690340129975d2cc0b465a6a"

In [ ]:
%pip install -qU langchain-deepseek
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.4 MB/s eta 0:00:00


# Google drive integration and data loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U gdown

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1IhTy0VlkN6SEafeF3i8THixz7flOe_sl -O ALLAPIS --quiet

In [ ]:

# Define the path to the CSV file inside your custom folder
csv_path = '/content/ALLAPIS/CEAS_08.csv'

# Check if file exists and load it
if os.path.exists(csv_path):
    print("✅ File found! Reading the CSV now...\n")
    df = pd.read_csv(csv_path)
   # display(df.head(10))  # Display first 10 rows
else:
    print("❌ File not found at:", csv_path)


✅ File found! Reading the CSV now...



In [ ]:
# Locating the file in the drive
import os

def find_folder_path(folder_name, base_path='/content/'):
    for root, dirs, files in os.walk(base_path):
        if folder_name in dirs:
            return os.path.join(root, folder_name)
    return None




In [ ]:
# Printing the path of the file
folder_name_to_search = '04 Manikanth and Anjireddy'
# path = find_folder_path(folder_name_to_search)
path = "/content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy"
print(path)

/content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy


In [ ]:

from google.colab import data_table
from vega_datasets import data

data_table.enable_dataframe_formatter()
df.head(3)


,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1


# preprocessing the data like remove duplicates and missing values

In [ ]:
#  Step 1: Remove duplicate rows
df = df.drop_duplicates()
from google.colab import data_table
from IPython.display import HTML
# df1 = df.head(3)
# data_table.DataTable(df)
data_table.DataTable(df.head(2), include_index=True, num_rows_per_page=5)




,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1


In [ ]:
# Step 2: Remove rows with any missing (NaN) values
df = df.dropna()
data_table.DataTable(df.head(2), include_index=True, num_rows_per_page=5)

# print(df1.head(3))



,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1


In [ ]:
# Optional: Reset index for clean row numbering
df.reset_index(drop=True, inplace=True)
data_table.DataTable(df.head(2), include_index=True, num_rows_per_page=5)
# print(df1.head(3))



,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1


In [ ]:
# startingRange = int(input("Enter the starting range: "))
# EndingRange = int(input("Enter the ending range: "))
# show_range(df, startingRange, EndingRange)  # Shows rows x to y
from google.colab import data_table
from vega_datasets import data

data_table.enable_dataframe_formatter()
df.head(10)


,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1
5,Caroline Aragon <dwthaidomainnamesm@thaidomain...,user7-ext5@gvc.ceas-challenge.cc,"Wed, 06 Aug 2008 05:31:22 +0600",From Caroline Aragon,\n\n\n\n\nYo wu urS mo ou go rc ebo eForM rgi ...,1,0
6,Replica Watches <jhorton@thebakercompanies.com>,user2.10@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 21:44:01 +0000",Replica Watches,We have fake Swiss Men's and Ladie's Replica \...,1,0
7,Daily Top 10 <acidirev_1972@tcwpg.com>,user2.3@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:41:14 -0300",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
8,qydlqcws-iacfym@issues.apache.org,xrh@spamassassin.apache.org,"Tue, 05 Aug 2008 15:31:03 -0800",[Bug 5780] URI processing turns uuencoded stri...,http://issues.apache.org/SpamAssassin/show_bug...,0,1
9,Daily Top 10 <orn|dent_1973@musicaedischi.it>,user7@gvc.ceas-challenge.cc,"Wed, 06 Aug 2008 00:31:38 +0100",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1


In [ ]:
# Define path to save the cleaned file
save_path = '/content/ALLAPIS/cleaned_data.csv'

# Save the cleaned DataFrame as CSV
df.to_csv(save_path, index=False)

print(f"✅ Cleaned data saved to: {save_path}")


✅ Cleaned data saved to: /content/ALLAPIS/cleaned_data.csv


# Extracting the data into header,body and header and body

## Data with labels

In [ ]:

#  Header-only DataFrame with label
df_header_label = df[['sender', 'receiver', 'date', 'subject', 'label']].copy()
data_table.DataTable(df_header_label.head(2), include_index=True, num_rows_per_page=5)







,sender,receiver,date,subject,label
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,1


In [ ]:
#  Body-only DataFrame with label
df_body_label = df[['body','label']].copy()
data_table.DataTable(df_body_label.head(10), include_index=True, num_rows_per_page=5)



,body,label
0,"Buck up, your troubles caused by small dimensi...",1
1,\nUpgrade your sex and pleasures with these te...,1
2,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1
3,Would anyone object to removing .so from this ...,0
4,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1
5,\n\n\n\n\nYo wu urS mo ou go rc ebo eForM rgi ...,1
6,We have fake Swiss Men's and Ladie's Replica \...,1
7,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1
8,http://issues.apache.org/SpamAssassin/show_bug...,0
9,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1


In [ ]:
# getting header and body  with label
df_header_body_label = df[['sender', 'receiver', 'date', 'subject', 'body','label']].copy()
data_table.DataTable(df_header_body_label.head(10), include_index=True, num_rows_per_page=5)



,sender,receiver,date,subject,body,label
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1
5,Caroline Aragon <dwthaidomainnamesm@thaidomain...,user7-ext5@gvc.ceas-challenge.cc,"Wed, 06 Aug 2008 05:31:22 +0600",From Caroline Aragon,\n\n\n\n\nYo wu urS mo ou go rc ebo eForM rgi ...,1
6,Replica Watches <jhorton@thebakercompanies.com>,user2.10@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 21:44:01 +0000",Replica Watches,We have fake Swiss Men's and Ladie's Replica \...,1
7,Daily Top 10 <acidirev_1972@tcwpg.com>,user2.3@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:41:14 -0300",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1
8,qydlqcws-iacfym@issues.apache.org,xrh@spamassassin.apache.org,"Tue, 05 Aug 2008 15:31:03 -0800",[Bug 5780] URI processing turns uuencoded stri...,http://issues.apache.org/SpamAssassin/show_bug...,0
9,Daily Top 10 <orn|dent_1973@musicaedischi.it>,user7@gvc.ceas-challenge.cc,"Wed, 06 Aug 2008 00:31:38 +0100",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1


In [ ]:

# Define your full target path
submission_folder = path
print(submission_folder)
deepseek_llm_path = os.path.join(submission_folder,'001 Submission','All APIs',  'deepseek_llm', 'raw_data','Data_with_label')

#  Create the directory structure
os.makedirs(deepseek_llm_path, exist_ok=True)

# Save the 3 labeled DataFrames to the new location

# 1. Header with label
df_header_label.to_csv(os.path.join(deepseek_llm_path, 'header_with_label.csv'), index=False)

# 2. Body with label
df_body_label.to_csv(os.path.join(deepseek_llm_path, 'body_with_label.csv'), index=False)

# 3. Combined Header + Body with label
df_header_body_label.to_csv(os.path.join(deepseek_llm_path, 'header_body_with_label.csv'), index=False)

print(f"✅ All files saved successfully to: {deepseek_llm_path}")


/content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy
✅ All files saved successfully to: /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/raw_data/Data_with_label


## Data without label

In [ ]:
#. Header-only DataFrame with no label
df_header_label = df[['sender', 'receiver', 'date', 'subject']].copy()
data_table.DataTable(df_header_label.head(2), include_index=True, num_rows_per_page=5)



,sender,receiver,date,subject
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson


In [ ]:
# Body-only DataFrame with no label
df_body_label = df[['body']].copy()
data_table.DataTable(df_body_label.head(2), include_index=True, num_rows_per_page=5)


,body
0,"Buck up, your troubles caused by small dimensi..."
1,\nUpgrade your sex and pleasures with these te...


In [ ]:
#  Combined header + body DataFrame with no label
df_header_body_label = df[['sender', 'receiver', 'date', 'subject', 'body']].copy()
data_table.DataTable(df_header_body_label.head(2), include_index=True, num_rows_per_page=5)


,sender,receiver,date,subject,body
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi..."
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...


In [ ]:
# Define your full target path
submission_folder = path
deepseek_llm_path = os.path.join(submission_folder,'001 Submission','All APIs',  'deepseek_llm', 'prompt_data','All_data')

# Create the directory structure
os.makedirs(deepseek_llm_path, exist_ok=True)

# Save the 3 labeled DataFrames to the new location

# 1. Header with label
df_header_label.to_csv(os.path.join(deepseek_llm_path, 'header_without_label.csv'), index=False)

# 2. Body with label
df_body_label.to_csv(os.path.join(deepseek_llm_path, 'body_without_label.csv'), index=False)

# 3. Combined Header + Body with label
df_header_body_label.to_csv(os.path.join(deepseek_llm_path, 'header_body_without_label.csv'), index=False)

print(f"✅ All files saved successfully to: {deepseek_llm_path}")


✅ All files saved successfully to: /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/prompt_data/All_data


## create sample 75 rows

### 75 Data with labels

In [ ]:

#  Define input and output folder paths
input_path =  os.path.join(path,'001 Submission','All APIs',  'deepseek_llm', 'raw_data','Data_with_label')
#  '/content/drive/MyDrive/outPut_Folder/gpt_llm/raw_data/Data_with_label'
output_path = os.path.join(path,'001 Submission','All APIs',  'deepseek_llm', 'raw_data','sample_75_with_labels')
# '/content/drive/MyDrive/outPut_Folder/gpt_llm/raw_data/sample_75_with_labels'
os.makedirs(output_path, exist_ok=True)  # Create output folder if not exists

# Define filenames and new names for output
files_to_sample = {
    'header_with_label.csv': 'header_sample_75_with_label.csv',
    'body_with_label.csv': 'body_sample_75_with_label.csv',
    'header_body_with_label.csv': 'header_body_sample_75_with_label.csv'
}

#  Load each file, extract 75 rows, and save to new location
for input_file, output_file in files_to_sample.items():
    full_input_path = os.path.join(input_path, input_file)
    full_output_path = os.path.join(output_path, output_file)

    # Load the full file
    df = pd.read_csv(full_input_path)

    # Extract the first 75 rows
    df_sample = df.head(75)

    # Save to the new folder
    df_sample.to_csv(full_output_path, index=False)
    print(f"✅ Saved first 75 rows of {input_file} → {full_output_path}")


✅ Saved first 75 rows of header_with_label.csv → /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/raw_data/sample_75_with_labels/header_sample_75_with_label.csv
✅ Saved first 75 rows of body_with_label.csv → /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/raw_data/sample_75_with_labels/body_sample_75_with_label.csv
✅ Saved first 75 rows of header_body_with_label.csv → /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/raw_data/sample_75_with_labels/header_body_sample_75_with_label.csv


### 75 data without labels

In [ ]:

# Define input and output folder paths
input_path = os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'prompt_data','All_data')
# '/content/drive/MyDrive/outPut_Folder/gpt_llm/prompt_data/All_data'
output_path = os.path.join(path,'001 Submission','All APIs',  'deepseek_llm', 'prompt_data','sample_75_without_labels')
# '/content/drive/MyDrive/outPut_Folder/gpt_llm/prompt_data/sample_75_without_labels'
os.makedirs(output_path, exist_ok=True)  # Create output folder if not exists

# Define filenames and new names for output
files_to_sample = {
    'header_without_label.csv': 'header_sample_75_without_label.csv',
    'body_without_label.csv': 'body_sample_75_without_label.csv',
    'header_body_without_label.csv': 'header_body_sample_75_without_label.csv'
}

# Load each file, extract 75 rows, and save to new location
for input_file, output_file in files_to_sample.items():
    full_input_path = os.path.join(input_path, input_file)
    full_output_path = os.path.join(output_path, output_file)

    # Load the full file
    df = pd.read_csv(full_input_path)

    # Extract the first 75 rows
    df_sample = df.head(75)

    # Save to the new folder
    df_sample.to_csv(full_output_path, index=False)
    print(f"✅ Saved first 75 rows of {input_file} → {full_output_path}")


✅ Saved first 75 rows of header_without_label.csv → /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/prompt_data/sample_75_without_labels/header_sample_75_without_label.csv
✅ Saved first 75 rows of body_without_label.csv → /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/prompt_data/sample_75_without_labels/body_sample_75_without_label.csv
✅ Saved first 75 rows of header_body_without_label.csv → /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/prompt_data/sample_75_without_labels/header_body_sample_75_without_label.csv


### printing the dataset

In [ ]:

# Load header sample
df_header_sample = pd.read_csv(os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'prompt_data','sample_75_without_labels','header_sample_75_without_label.csv'))
# '/content/drive/MyDrive/outPut_Folder/gpt_llm/prompt_data/sample_75_without_labels/header_sample_75_without_label.csv

# Load body sample
df_body_sample = pd.read_csv(os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'prompt_data','sample_75_without_labels','body_sample_75_without_label.csv'))
                            #  '/content/drive/MyDrive/outPut_Folder/gpt_llm/prompt_data/sample_75_without_labels/body_sample_75_without_label.csv'

# Load combined header+body sample
df_combined_sample = pd.read_csv(os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'prompt_data','sample_75_without_labels','header_body_sample_75_without_label.csv'))
                                #  '/content/drive/MyDrive/outPut_Folder/gpt_llm/prompt_data/sample_75_without_labels/header_body_sample_75_without_label.csv'


In [ ]:
df_header_sample.head(10)


,sender,receiver,date,subject
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo
5,Caroline Aragon <dwthaidomainnamesm@thaidomain...,user7-ext5@gvc.ceas-challenge.cc,"Wed, 06 Aug 2008 05:31:22 +0600",From Caroline Aragon
6,Replica Watches <jhorton@thebakercompanies.com>,user2.10@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 21:44:01 +0000",Replica Watches
7,Daily Top 10 <acidirev_1972@tcwpg.com>,user2.3@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:41:14 -0300",CNN.com Daily Top 10
8,qydlqcws-iacfym@issues.apache.org,xrh@spamassassin.apache.org,"Tue, 05 Aug 2008 15:31:03 -0800",[Bug 5780] URI processing turns uuencoded stri...
9,Daily Top 10 <orn|dent_1973@musicaedischi.it>,user7@gvc.ceas-challenge.cc,"Wed, 06 Aug 2008 00:31:38 +0100",CNN.com Daily Top 10


In [ ]:
df_body_sample.head(50)

,body
0,"Buck up, your troubles caused by small dimensi..."
1,\nUpgrade your sex and pleasures with these te...
2,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...
3,Would anyone object to removing .so from this ...
4,\nWelcomeFastShippingCustomerSupport\nhttp://7...
5,\n\n\n\n\nYo wu urS mo ou go rc ebo eForM rgi ...
6,We have fake Swiss Men's and Ladie's Replica \...
7,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...
8,http://issues.apache.org/SpamAssassin/show_bug...
9,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...


In [ ]:
df_combined_sample.head(2)

,sender,receiver,date,subject,body
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi..."
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...


In [ ]:
def create_zeroshot_prompt(row, email_type="header"):

    if email_type == "header":
        return f"""
Classify this email header as 'Phishing' or 'Legitimate'. Then briefly explain why.

Sender: {row['sender']}
Receiver: {row['receiver']}
Subject: {row['subject']}
Date: {row['date']}

Format: Prediction|Reason
"""

    elif email_type == "body":
        return f"""
Classify this email body as 'Phishing' or 'Legitimate'. Then briefly explain why.

Body: {row['body']}

Format: Prediction|Reason
"""

    elif email_type == "combined":
        return f"""
Classify this email (header and body) as 'Phishing' or 'Legitimate'. Then briefly explain why.

Header:
Sender: {row['sender']}
Receiver: {row['receiver']}
Subject: {row['subject']}
Date: {row['date']}

Body:
{row['body']}
use the below format for prediction and reason

Format:Phishing or Legitimate|Reason
"""

    else:
        raise ValueError("❌ Invalid email_type specified.")


In [ ]:
# ✅ Sends a prompt to DeepSeek and returns clean Prediction & Reason
def analyze_email_with_deepseek(prompt):
    try:
        # 🔹 Send prompt to DeepSeek model (already initialized above)
        response = llm.invoke(prompt)

        # 🔹 Get the reply content
        reply = response.content.strip().replace("\n", " ").replace("Answer:", "").replace("Prediction:", "").replace("*", "")
        print(reply)

        # ⚠️ Risky: split the response using '|'

        parts = reply.split("|")
        # print("p")
        # print(parts)
        # print(parts.str.contains("Phishing"))
        # if parts[0].contains("Phishing"):
        #     parts[0] = "Phishing"
        #     print("phishing")
        # elif parts[0].str.contains("Legitimate"):
        #     parts[0] = "Legitimate"
        # # print(parts
        # print(parts[0])


        # ✅ FIX #1: If only prediction is returned (no '|'), pad reason
        if len(parts) == 1:
            parts.append("No explanation provided.")


        # ✅ FIX #2: If too many '|', merge into single reason
        elif len(parts) > 2:
            parts = [parts[0], "|".join(parts[1:])]

        # ✅ Return as two clean strings in a Series
        return pd.Series([p.strip() for p in parts])

    except Exception as e:
        # ✅ FIX #3: Catch API/logic issues and return consistent error output
        return pd.Series(["Error", str(e)])


In [ ]:
# ✅ Processes a CSV file using DeepSeek and saves predictions to Excel
def process_sample_75_csv_with_deepseek(file_name, email_type):
    import os
    import pandas as pd

    # 🔹 Input file location
    input_path = os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'prompt_data','sample_75_without_labels',file_name)
        # input_path = f'/content/drive/MyDrive/outPut_Folder/gpt_llm/prompt_data/sample_75_without_labels/{file_name}'
    # Define output folder location
    output_folder = os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'output_daata','zeroshot')
    #  '/content/drive/MyDrive/outPut_Folder/gpt_llm/output_daata/zeroshot'
    os.makedirs(output_folder, exist_ok=True)

    # 🔹 Output filename
    output_file = file_name.replace('.csv', '_deepseek_predicted.xlsx')
    output_path = os.path.join(output_folder, output_file)

    # 🔹 Load the input file
    df = pd.read_csv(input_path)

    # 🔹 Apply DeepSeek to each row
    df[["Prediction", "Reason"]] = df.apply(
        lambda row: analyze_email_with_deepseek(create_zeroshot_prompt(row, email_type)),
        axis=1
    )

    # 🔹 Add numeric prediction label
    df["prediction_label"] = df["Prediction"].map(
            lambda x: 1 if isinstance(x, str) and "phishing" in x.strip().lower() else 0
            )
    # df["prediction_label"] = df["Prediction"].map(
    #     lambda x: 1 if isinstance(x, str) and x.strip().lower().contains("phishing") else 0
    # )


    # 🔹 Reorder prediction_label right after Prediction
    cols = df.columns.tolist()
    if "Prediction" in cols and "Reason" in cols and "prediction_label" in cols:
        pred_idx = cols.index("Prediction")
        cols.insert(pred_idx + 1, cols.pop(cols.index("prediction_label")))
        df = df[cols]

    # 🔹 Save to Excel
    df.to_excel(output_path, index=False)
    print(f"✅ DeepSeek processed: {file_name} → Saved to {output_path}")


In [ ]:
# For header-only email dataset
process_sample_75_csv_with_deepseek("header_sample_75_without_label.csv", email_type="header")

# For body-only email dataset
process_sample_75_csv_with_deepseek("body_sample_75_without_label.csv", email_type="body")

# For combined header + body dataset
process_sample_75_csv_with_deepseek("header_body_sample_75_without_label.csv", email_type="combined")


 Phishing|Reason: The sender's email domain 'iworld.de' does not match the subject's urgency and tone ("Never agree to be a loser"), which is typical of phishing attempts. Additionally, the receiver's domain 'gvc.ceas-challenge.cc' appears unusual and potentially fabricated, further suggesting a phishing attempt. The mismatch between the sender's name and email address (Young Esposito vs. 'Young@iworld.de') also raises suspicion.
 Phishing|Reason: The sender's email address ('ipline's1983@icable.ph') appears suspicious with an unusual username and domain. The subject line ('Befriend Jenna Jameson') is attention-grabbing and unrelated to typical professional or personal correspondence, which is a common tactic in phishing emails to lure recipients. Additionally, the receiver's domain ('gvc.ceas-challenge.cc') seems non-standard, possibly indicating a testing or malicious environment. The combination of these factors suggests a phishing attempt.
 Phishing   Reason: The sender's email add

#Fewshot

In [ ]:
# ✅ Pre-defined examples (few-shot) to "teach" the model before each real classification
FEW_SHOT_EXAMPLES = {
    "header": [
        {
            "sender": "admin@securebank.com",
            "receiver": "user123@gmail.com",
            "subject": "Important Account Notification",
            "date": "2024-11-05",
            "label": "Phishing",
            "reason": "The sender's email is suspicious and urges action."
        },
        {
            "sender": "newsletter@trustedsource.com",
            "receiver": "user123@gmail.com",
            "subject": "Monthly Newsletter - November",
            "date": "2024-11-01",
            "label": "Legitimate",
            "reason": "It’s a regular newsletter from a known sender."
        }
    ],
    "body": [
        {
            "body": "Your account has been locked. Click here to verify immediately: http://scam-site.com",
            "label": "Phishing",
            "reason": "Urgent tone and suspicious link included."
        },
        {
            "body": "Here is your delivery confirmation for order #12345. Thank you for shopping with us!",
            "label": "Legitimate",
            "reason": "Normal e-commerce confirmation message."
        }
    ],
    "combined": [
        {
            "sender": "alerts@paypal-security.com",
            "receiver": "user@gmail.com",
            "subject": "Verify your PayPal account now!",
            "date": "2024-11-08",
            "body": "We noticed suspicious activity. Click below to confirm.",
            "label": "Phishing",
            "reason": "Suspicious sender and urgent call-to-action."
        },
        {
            "sender": "support@github.com",
            "receiver": "user@gmail.com",
            "subject": "New login detected",
            "date": "2024-11-01",
            "body": "We noticed a new login from a different location. If this was you, no further action is required.",
            "label": "Legitimate",
            "reason": "Clear explanation, no suspicious links, from a known service."


        }
    ]
}


In [ ]:
def create_few_shot_prompt(row, email_type="header"):
    examples = FEW_SHOT_EXAMPLES[email_type]
    prompt = "You are an email security classifier. Learn from the examples below.\n\n   example output : Phishing or legitimate | Reason"


    # Add compact few-shot examples
    for ex in examples:
        if email_type == "header":
            prompt += (
                f"Sender: {ex['sender']}, Subject: {ex['subject']}, Date: {ex['date']}\n"
                f"Answer: {ex['label']} | {ex['reason']}\n\n"
            )
        elif email_type == "body":
            prompt += (
                f"Body: {ex['body']}\n"
                f"Answer: {ex['label']} | {ex['reason']}\n\n"
            )
        elif email_type == "combined":
            prompt += (
                f"Sender: {ex['sender']}, Subject: {ex['subject']}, Date: {ex['date']}, Body: {ex['body']}\n"
                f"Answer: {ex['label']} | {ex['reason']}\n\n"
            )

    # Now add the row to classify
    if email_type == "header":
        prompt += (
            f"Sender: {row['sender']}, Subject: {row['subject']}, Date: {row['date']}\n"
            f"Answer:"
        )
    elif email_type == "body":
        prompt += (
            f"Body: {row['body']}\n"
            f"Answer:"
        )
    elif email_type == "combined":
        prompt += (
            f"Sender: {row['sender']}, Subject: {row['subject']}, Date: {row['date']}, Body: {row['body']}\n"
            f"Answer:"
        )

    return prompt


In [ ]:
def analyze_email_few_shot(prompt):
    try:
        # 🔹 Send prompt to DeepSeek model (already initialized above)
        response = llm.invoke(prompt)
        # 🔹 Get the reply content
        reply = response.content.strip().replace("\n", " ").replace("Answer:", "").replace(" ", "")
        print(reply)

        # ⚠️ Risky: split the response using '|'
        parts = reply.split("|")


        # ✅ FIX #1: If only prediction is returned (no '|'), pad reason
        if len(parts) == 1:
            parts.append("No explanation provided.")

        # ✅ FIX #2: If too many '|', merge into single reason
        elif len(parts) > 2:
            parts = [parts[0], "|".join(parts[1:])]

        # ✅ Return as two clean strings in a Series
        return pd.Series([p.strip() for p in parts])
    except Exception as e:
        return pd.Series(["Error", str(e)])


In [ ]:

def process_sample_75_few_shot(file_name, email_type):
    import os
    input_path = os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'prompt_data','sample_75_without_labels',file_name)
    # f'/content/drive/MyDrive/outPut_Folder/gpt_llm/prompt_data/sample_75_without_labels/{file_name}'
    output_folder = os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'output_daata','fewshot')
    #  '/content/drive/MyDrive/outPut_Folder/gpt_llm/output_daata/fewshot'
    os.makedirs(output_folder, exist_ok=True)

    output_file = file_name.replace('.csv', '_fewshot_predicted.xlsx')
    output_path = os.path.join(output_folder, output_file)

    # Load input data
    df = pd.read_csv(input_path)

    # Apply Deepseeek to each row using few-shot prompt
    df[['Prediction', 'Reason']] = df.apply(
        lambda row: analyze_email_few_shot(create_few_shot_prompt(row, email_type)),
        axis=1)


    # Convert textual label to numeric
    # df["prediction_label"] = df["Prediction"].map(lambda x: 1 if x.strip().lower() == "phishing" else 0)
    df["prediction_label"] = df["Prediction"].map(
            lambda x: 1 if isinstance(x, str) and "phishing" in x.strip().lower() else 0
            )

    # Reorder prediction_label
    cols = df.columns.tolist()
    if "Prediction" in cols and "Reason" in cols and "prediction_label" in cols:
        pred_idx = cols.index("Prediction")
        cols.insert(pred_idx + 1, cols.pop(cols.index("prediction_label")))
        df = df[cols]

    # Save to Excel
    df.to_excel(output_path, index=False)
    print(f"✅ Few-shot processed and saved → {output_path}")


In [ ]:
process_sample_75_few_shot("header_sample_75_without_label.csv", "header")
process_sample_75_few_shot("body_sample_75_without_label.csv", "body")
process_sample_75_few_shot("header_body_sample_75_without_label.csv", "combined")


Phishing|Thesender'snameandemailaresuspicious,andthesubjectlineisaggressiveandunprofessional.
Phishing|Thesender'semailaddressissuspiciousandthesubjectlineisinappropriateandunrelatedtotypicalprofessionalorpersonalcorrespondence.
Phishing|Thesender'semailaddressdoesnotmatchtheclaimedsource(CNN)andcontainsunusualcharacters.Thedateformatisalsooutdatedandsuspicious.
Phishing|Thesender'semailaddressissuspiciousandthesubjectcontainstechnicaljargonthatmaybeusedtodeceivetherecipient.
Phishing|Thesender'semailandsubjectaresuspicious,andtheemailappearstobepromotingadubiousoffer.
Phishing|Thesender'semailaddressissuspicious(containsrandomcharacters)andthesubjectisvague,whicharecommontraitsofphishingemails.
Phishing|Theemailispromotingreplicawatches,whichisacommontacticinphishingscams,andthesender'semaildoesnotmatchthecontent.
Phishing|Thesender'semailaddressdoesnotmatchtheclaimedsource(CNN)andappearssuspicious.
Legitimate|Theemailisfromaknowndomain(issues.apache.org)andthesubjectpertainstoaspecif

#Chain of thought

In [ ]:
def analyze_email_chain_of_thought(prompt):

    try:
        # 🔹 Send prompt to DeepSeek model (already initialized above)
        response = llm.invoke(prompt)

        # 🔹 Get the reply content
        reply = response.content.strip().replace("\n", " ").replace("Answer:", "").replace(" ", "")
        print(reply)

        # ⚠️ Risky: split the response using '|'
        parts = reply.split("|")

        # ✅ FIX #1: If only prediction is returned (no '|'), pad reason
        if len(parts) == 1:
            parts.append("No explanation provided.")

        # ✅ FIX #2: If too many '|', merge into single reason
        elif len(parts) > 2:
            parts = [parts[0], "|".join(parts[1:])]

        # ✅ Return as two clean strings in a Series
        return pd.Series([p.strip() for p in parts])

    except Exception as e:
        # ❌ If Deepseek call fails, return error message for logging or review
        return pd.Series(["Error", str(e)])


In [ ]:
def create_chain_of_thought_prompt(row, email_type="header"):

    if email_type == "header":
        return f"""
You are a cybersecurity expert. Decide if this email header is phishing or legitimate.
Think about sender trust, subject urgency, and time of day.

Sender: {row['sender']}
Receiver: {row['receiver']}
Subject: {row['subject']}
Date: {row['date']}

Answer format: Legitimate or Phishing | Brief Reason
"""

    elif email_type == "body":
        return f"""
You are a cybersecurity expert. Decide if this email body is phishing or legitimate.
Think about urgency, suspicious links, requests for sensitive information, and tone.

Body: {row['body']}

Answer format: Legitimate or Phishing | Brief Reason
"""

    elif email_type == "combined":
        return f"""
You're a cybersecurity analyst. Decide if this full email is phishing or legitimate.
Consider sender trust, subject line, suspicious content, links, urgency, and tone of the body.

Sender: {row['sender']}
Receiver: {row['receiver']}
Subject: {row['subject']}
Date: {row['date']}
Body: {row['body']}

Answer format: Legitimate or Phishing | Brief Reason
"""

    else:
        raise ValueError("❌ Invalid email_type passed to prompt generator.")


In [ ]:
# ✅ Loads a sample_75 CSV file, applies CoT prompting, and saves results as Excel
def process_chain_of_thought(file_name, email_type="header"):
    # Define paths
    input_path = os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'prompt_data','sample_75_without_labels',file_name)
    #  f'/content/drive/MyDrive/outPut_Folder/gpt_llm/prompt_data/sample_75_without_labels/{file_name}'
    output_folder = os.path.join(path,'001 Submission','All APIs', 'deepseek_llm', 'output_daata','chainofthought')
    #  '/content/drive/MyDrive/outPut_Folder/gpt_llm/output_daata/chainofthought'
    os.makedirs(output_folder, exist_ok=True)

    output_file = file_name.replace('.csv', '_cot_predicted.xlsx')
    output_path = os.path.join(output_folder, output_file)

    # Load input CSV into DataFrame
    df = pd.read_csv(input_path)

    # Generate CoT prompt per email
    df[['Prediction', 'Reason']] = df.apply(
        lambda row: analyze_email_chain_of_thought(create_chain_of_thought_prompt(row, email_type)),
        axis=1
    )

    # Map textual prediction to numeric label (1 = phishing, 0 = legitimate)
    df["prediction_label"] = df["Prediction"].map(
            lambda x: 1 if isinstance(x, str) and "phishing" in x.strip().lower() else 0
            )
    # df["prediction_label"] = df["Prediction"].apply(
    #     lambda x: 1 if isinstance(x, str) and "phishing" in x.lower() else 0
    # )

    # Reorder columns (optional)
    cols = df.columns.tolist()
    if "Prediction" in cols and "Reason" in cols and "prediction_label" in cols:
        pred_idx = cols.index("Prediction")
        cols.insert(pred_idx + 1, cols.pop(cols.index("prediction_label")))
        df = df[cols]

    # Save final DataFrame as Excel
    df.to_excel(output_path, index=False)

    # ✅ Confirm completion
    print(f"✅ CoT analysis complete for '{email_type}' emails. File saved at: {output_path}")


In [ ]:
process_chain_of_thought("header_sample_75_without_label.csv", "header")
process_chain_of_thought("body_sample_75_without_label.csv", "body")
process_chain_of_thought("header_body_sample_75_without_label.csv", "combined")


Phishing|Thesender'sdomain"iworld.de"doesnotappeartobealegitimateorwell-knownorganization,andthesubjectline"Neveragreetobealoser"isvagueandcreatesunnecessaryurgencywithoutclearcontext.Additionally,theemailwassentlateintheday(4:31PM),whichisacommontacticinphishingattemptstocatchrecipientsoffguard.Thecombinationofthesefactorssuggestsaphishingattempt.
Phishing|Thesender'semailaddress(ipline's1983@icable.ph)appearssuspiciouswithanunusualusernameanddomain.Thesubject"BefriendJennaJameson"isirrelevantandlikelyclickbait,whichisacommontacticinphishingemails.Additionally,thetimeofday(evening)andlackofprofessionalcontextfurthersuggestthisisnotalegitimatecommunication.
Phishing|Thesender'semaildomain"universalnet.psi.br"doesnotmatchtheexpecteddomainforCNN(e.g.,cnn.com).ThesubjectlinemimicsalegitimateCNNnewsletter,butthesender'saddressissuspicious.Additionally,thetimezoneoffset"-1200"isinvalid(timezonesrangefrom-12to+14),indicatingpossibleheadermanipulation.Theseinconsistenciessuggestaphishingattem

In [ ]:
import pandas as pd

# ✅ Function to summarize predictions in any result file
def summarize_predictions(filepath, prompt_type, email_type):
    try:
        # 🔹 Load the prediction result file
        df = pd.read_excel(filepath)

        # 🔹 Clean + Normalize the predictions
        df["Prediction"] = df["Prediction"].str.strip().str.lower()

        # 🔹 Count the number of phishing and legitimate predictions
        phishing_count = (df["Prediction"] == "phishing").sum()
        legit_count = (df["Prediction"] == "legitimate").sum()
        total = len(df)

        # ✅ Print a summary
        print(f"\n📂 {prompt_type.upper()} | 📧 {email_type.upper()}")
        print(f"📊 Total Emails: {total}")
        print(f"🛑 Phishing: {phishing_count}")
        print(f"✅ Legitimate: {legit_count}")
        print("-" * 40)

    except Exception as e:
        print(f"❌ Error processing {filepath}: {e}")


In [ ]:
# 🔹 Base path
base_path = os.path.join(path,'001 Submission','All APIs',  'deepseek_llm', 'output_daata')

# 🔹 File and config list (path, type)
files_info = [
    # Zero-shot
    (f"{base_path}/zeroshot/header_sample_75_without_label_predicted.xlsx", "zero-shot", "header"),
    (f"{base_path}/zeroshot/body_sample_75_without_label_predicted.xlsx", "zero-shot", "body"),
    (f"{base_path}/zeroshot/header_body_sample_75_without_label_predicted.xlsx", "zero-shot", "combined"),

    # Few-shot
    (f"{base_path}/fewshot/header_sample_75_without_label_fewshot_predicted.xlsx", "few-shot", "header"),
    (f"{base_path}/fewshot/body_sample_75_without_label_fewshot_predicted.xlsx", "few-shot", "body"),
    (f"{base_path}/fewshot/header_body_sample_75_without_label_fewshot_predicted.xlsx", "few-shot", "combined"),

    # Chain-of-Thought
    (f"{base_path}/chainofthought/header_sample_75_without_label_cot_predicted.xlsx", "chain-of-thought", "header"),
    (f"{base_path}/chainofthought/body_sample_75_without_label_cot_predicted.xlsx", "chain-of-thought", "body"),
    (f"{base_path}/chainofthought/header_body_sample_75_without_label_cot_predicted.xlsx", "chain-of-thought", "combined"),
]

# Run summaries
for file_path, prompt_type, email_type in files_info:
    summarize_predictions(file_path, prompt_type, email_type)


❌ Error processing /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/output_daata/zeroshot/header_sample_75_without_label_predicted.xlsx: [Errno 2] No such file or directory: '/content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/output_daata/zeroshot/header_sample_75_without_label_predicted.xlsx'
❌ Error processing /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/output_daata/zeroshot/body_sample_75_without_label_predicted.xlsx: [Errno 2] No such file or directory: '/content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/output_daata/zeroshot/body_sample_75_without_label_predicted.xlsx'
❌ Error processing /content/drive/MyDrive/002 SP 25 Project/04 Manikanth and Anjireddy/001 Submission/All APIs/deepseek_llm/output_daata/zeroshot/header_body_sample_75_without_label_predicted.xl

# End of code